# **Trabalho \#2**:  Aprendizado de Máquina e Reconhecimento de Padrões MC886/MO444
Instituto de Computação (IC/UNICAMP), 2ᵒ Semestre, 2024

Profa. Sandra Avila, 2024s2

Trabalho proposto por Caio Petrucci Rosa (PAD), Cesar Guedes Carneiro (PED) e Sandra Avila.



In [2]:
# TODO: RA & Nome
print('231081: ' + 'Pedro da Rosa Pinheiro')
print('225334: ' + 'Jonatas de Sousa Santos')

231081: Pedro da Rosa Pinheiro
225334: Jonatas de Sousa Santos


## Objetivo

Explore os modelos de **regressão linear** e **regressão logística** e apresente o melhor modelo que você encontrar para o problema, evitando o *overfitting*. Em particular, realize predições acerca do salário dos profissionais da área de dados no Brasil, com base nos dados da pesquisa State of Data Brazil 2023, levando em consideração variáveis como perfil demográfico, formação, experiência e atuação no setor.


---

## Base de Dados

A base de dados deste trabalho foi criado a partir de dados disponibilizados em [State of Data Brazil 2023](https://www.kaggle.com/datasets/datahackers/state-of-data-brazil-2023). Os dados refletem a pesquisa realizada com profissionais da área de Ciência de Dados no Brasil. As respostas foram coletadas em 2023 e a base de dados contém uma ampla variedade de perguntas sobre o perfil demográfico, situação profissional e conhecimentos técnicos dessas pessoas.

**Dicionário de dados:**

* **id**: Identificador único da pessoa participante;

* **idade**: Idade da pessoa participante;

* **genero**: Gênero da pessoa participante;

* **cor_raca**: Cor/raça/etnia da pessoa participante;

* **pcd**: Se a pessoa participante possui alguma deficiência (PCD);

* **estado_residencia**: Estado onde a pessoa participante mora;

* **mudou_estado**: Se a pessoa participante mudou de estado recentemente;

* **regiao_origem**: Região de origem da pessoa participante;

* **nivel_ensino**: Nível de ensino da pessoa participante;

* **area_formacao**: Área de formação acadêmica;

* **situacao_trabalho**: Situação atual de trabalho;

* **setor**: Setor em que a pessoa participante trabalha;

* **numero_funcionarios**: Número de funcionários na empresa onde a pessoa participante trabalha;

* **gestor**: Se a pessoa participante é gestor na empresa;

* **cargo**: Cargo da pessoa participante;

* **nivel_profissional**: Nível profissional da pessoa participante;

* **tempo_exp_dados**: Tempo de experiência na área de dados;

* **tempo_exp_ti**: Tempo de experiência prévia na área de TI ou Engenharia de Software antes de atuar com dados;

* **satisfacao_empresa**: Nível de satisfação da pessoa participante com sua empresa atual;

* **forma_trabalho_atual**: Forma de trabalho atual (presencial, remoto, híbrido);

* **atuacao_dia_a_dia**: Reflexão da atuação da pessoa participante no dia a dia em relação ao cargo formal;

* **fontes_dados_analise**: Fontes de dados já analisadas ou processadas pela pessoa participante no trabalho;

* **fontes_dados_uso**: Fontes de dados mais utilizadas pela pessoa participante no trabalho;

* **linguagens_trabalho**: Linguagens de programação utilizadas no trabalho;

* **linguagem_mais_usada**: Linguagem de programação mais utilizada no trabalho;

* **linguagem_preferida**: Linguagem de programação preferida da pessoa participante;

* **bancos_dados_trabalho**: Bancos de dados utilizados no trabalho;

* **cloud_preferida**: Cloud preferida da pessoa participante;

* **cloud_usada**: Cloud usada no trabalho;

* **ferramenta_bi_dia_a_dia**: Ferramenta de BI utilizada no dia a dia;

* **ferramenta_bi_preferida**: Ferramenta de BI preferida;

* **tipo_uso_ai**: Tipo de uso de AI Generativa e LLMs na empresa;

* **uso_chatgpt**: Se a pessoa participante utiliza ChatGPT ou LLMs no trabalho;

* **faixa_salarial**: Faixa salarial da pessoa participante;

* **salario**: Valor do salário da pessoa participante.


Você deve **respeitar a seguinte divisão de treino/teste**:

    train_data.csv
    test_data.csv

Lembre-se de criar **seu próprio conjunto de validação**, a partir um subconjunto do conjunto de treinamento, e utilize o conjunto de teste (a ser disponibilizado) apenas para inferência e relato do resultado final.

### Atividades

1. (0,4 pontos) Realize a limpeza e preparação dos dados de treinamento.

  > **Considere os seguintes fatores:**
  > - Existem outliers nos dados? Se sim, como você pretende tratá-los (removê-los, ajustá-los, ou usar uma abordagem alternativa)?
  > - Há valores ausentes? Se sim, como irá lidar com eles? (Ex.: exclusão de linhas, preenchimento por média/mediana/moda, ou outros métodos).
  > - Como você planeja tratar variáveis categóricas? (Ex.: one-hot encoding, label encoding, ou outra estratégia).

In [4]:
# TODO: Carregue e pré-processe (limpeza e preparação) os dados
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split

pd.options.display.max_columns = None


In [34]:
# Carregando dados
df_raw = pd.read_csv("train_data.csv")

# removendo features não relevantes
df = df_raw
df.drop(["id", "mudou_estado", "regiao_origem", "numero_funcionarios", "satisfacao_empresa", "forma_trabalho_atual", "fontes_dados_analise", "fontes_dados_uso", "linguagens_trabalho", "linguagem_mais_usada", "linguagem_preferida", "bancos_dados_trabalho", "cloud_preferida", "cloud_usada", "ferramenta_bi_dia_a_dia", "ferramenta_bi_preferida", "tipo_uso_ai", "uso_chatgpt"], axis=1, inplace=True)

# preenchendo valores NaN
df["nivel_profissional"].fillna(df["nivel_profissional"].value_counts().index[0], inplace=True)

# realizando one hot encoding
encoder = OneHotEncoder(sparse_output=False)

categorical_cols = ["genero", "cor_raca", "estado_residencia", "nivel_ensino", "area_formacao", "situacao_trabalho", "setor", "atuacao_dia_a_dia", "cargo"]
encoded = encoder.fit_transform(df[categorical_cols])
df_encoded = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(categorical_cols))

df = pd.concat([df, df_encoded], axis=1)
df.drop(categorical_cols, axis=1, inplace=True)


# renomeando features com nomes longos/não formatados
df.rename(columns={"setor_Tecnologia/Fábrica de Software": "setor_Tech/Fábrica",
                        "setor_Área da Saúde": "setor_Saude",
                        "setor_Área de Consultoria": "setor_Consultoria",
                        "atuacao_dia_a_dia_*Análise de Dados/BI:* Extrai e cruza dados unindo diferentes fontes da informação; analisa dados visando identificar padrões, gerar insights e levantar perguntas; desenvolve dashboards, relatórios e visualizações de dados em ferramentas de BI.": "atuacao_dia_a_dia_AD/BI",
                        "atuacao_dia_a_dia_*Ciência de Dados/Machine Learning/AI: *Desenha e executa experimentos com o objetivo de responder perguntas do negócio; desenvolve modelos preditivos e algoritmos de Machine Learning com o objetivo de otimizar e automatizar a tomada de decisão.": "atuacao_dia_a_dia_DS/ML/AI",
                        "atuacao_dia_a_dia_*Engenharia de Dados:* Modela soluções de arquitetura de dados; define modelagens de repositórios de dados (Data Lake, Data Warehouse, Data Lakehouse); desenvolve estratégias de aquisição de dados, recuperação de informação e pipelines de dados.": "atuacao_dia_a_dia_Engenharia",
                        "atuacao_dia_a_dia_Não atuo em nenhuma das frentes citadas.": "atuacao_dia_a_dia_Unknown",
                        "cor_raca_Prefiro não informar": "cor_raca_Unknown",
                        "estado_residencia_Acre (AC)": "estado_residencia_AC",
                        "estado_residencia_Alagoas (AL)": "estado_residencia_AL",
                        "estado_residencia_Amapá (AP)": "estado_residencia_AP",
                        "estado_residencia_Amazonas (AM)": "estado_residencia_AM",
                        "estado_residencia_Bahia (BA)": "estado_residencia_BA",
                        "estado_residencia_Ceará (CE)": "estado_residencia_CE",
                        "estado_residencia_Distrito Federal (DF)": "estado_residencia_DF",
                        "estado_residencia_Espírito Santo (ES)": "estado_residencia_ES",
                        "estado_residencia_Goiás (GO)": "estado_residencia_GO",
                        "estado_residencia_Maranhão (MA)": "estado_residencia_MA",
                        "estado_residencia_Mato Grosso (MT)": "estado_residencia_MT",
                        "estado_residencia_Mato Grosso do Sul (MS)": "estado_residencia_MS",
                        "estado_residencia_Minas Gerais (MG)": "estado_residencia_MG",
                        "estado_residencia_Paraná (PR)": "estado_residencia_PR",
                        "estado_residencia_Paraíba (PB)": "estado_residencia_PB",
                        "estado_residencia_Pará (PA)": "estado_residencia_PA",
                        "estado_residencia_Pernambuco (PE)": "estado_residencia_PE",
                        "estado_residencia_Piauí (PI)": "estado_residencia_PI",
                        "estado_residencia_Rio Grande do Norte (RN)": "estado_residencia_RN",
                        "estado_residencia_Rio Grande do Sul (RS)": "estado_residencia_RS",
                        "estado_residencia_Rio de Janeiro (RJ)": "estado_residencia_RJ",
                        "estado_residencia_Rondônia (RO)": "estado_residencia_RO",
                        "estado_residencia_Santa Catarina (SC)": "estado_residencia_SC",
                        "estado_residencia_Sergipe (SE)": "estado_residencia_SE",
                        "estado_residencia_São Paulo (SP)": "estado_residencia_SP",
                        "estado_residencia_Tocantins (TO)": "estado_residencia_TO",
                        "nivel_ensino_Doutorado ou PHD": "nivel_ensino_Doutorado/PHD",
                        "nivel_ensino_Estudante de Graduação": "nivel_ensino_Estudante",
                        "nivel_ensino_Não tenho graduação formal": "nivel_ensino_SemGrad",
                        "nivel_ensino_Prefiro não informar": "nivel_ensino_Unknown"}, inplace=True)

# realizando label encoding
label_encoder = LabelEncoder()
df["pcd"] = label_encoder.fit_transform(df["pcd"])

# realizando ordinal encoding
ordinal_encoder = OrdinalEncoder(categories=[["Júnior", "Pleno", "Sênior"]])
df["nivel_profissional"] = ordinal_encoder.fit_transform(df[["nivel_profissional"]])

ordinal_encoder = OrdinalEncoder(categories=[['Não tenho experiência na área de dados', 'Menos de 1 ano', 'de 1 a 2 anos', 'de 3 a 4 anos', 'de 4 a 6 anos', 'de 5 a 6 anos', 'de 7 a 10 anos', 'Mais de 10 anos']])
df["tempo_exp_dados"] = ordinal_encoder.fit_transform(df[["tempo_exp_dados"]])

ordinal_encoder = OrdinalEncoder(categories=[['Não tive experiência na área de TI/Engenharia de Software antes de começar a trabalhar na área de dados', 'Menos de 1 ano', 'de 1 a 2 anos', 'de 3 a 4 anos', 'de 5 a 6 anos', 'de 7 a 10 anos', 'Mais de 10 anos']])
df["tempo_exp_ti"] = ordinal_encoder.fit_transform(df[["tempo_exp_ti"]])

ordinal_encoder = OrdinalEncoder(categories=[['de R$ 101/mês a R$ 2.000/mês', 'Menos de R$ 1.000/mês', 'de R$ 1.001/mês a R$ 2.000/mês', 'de R$ 2.001/mês a R$ 3.000/mês', 'de R$ 3.001/mês a R$ 4.000/mês', 'de R$ 4.001/mês a R$ 6.000/mês', 'de R$ 6.001/mês a R$ 8.000/mês', 'de R$ 8.001/mês a R$ 12.000/mês', 'de R$ 12.001/mês a R$ 16.000/mês', 'de R$ 16.001/mês a R$ 20.000/mês', 'de R$ 20.001/mês a R$ 25.000/mês', 'de R$ 25.001/mês a R$ 30.000/mês', 'de R$ 30.001/mês a R$ 40.000/mês', 'Acima de R$ 40.001/mês']])
df["faixa_salarial"] = ordinal_encoder.fit_transform(df[["faixa_salarial"]])

# normalizando dados
min_max_scaler = MinMaxScaler()
df_temp = df
df = pd.DataFrame(min_max_scaler.fit_transform(df.drop(["salario", "faixa_salarial"], axis=1)), index=df.index, columns=df.drop(["salario", "faixa_salarial"], axis=1).columns)
df["salario"] = df_temp["salario"]
df["faixa_salarial"] = df_temp["faixa_salarial"]

print(df.head(10))
print(df.shape)



      idade  pcd  gestor  nivel_profissional  tempo_exp_dados  tempo_exp_ti  \
0  0.211538  0.0     1.0                 1.0         0.857143      0.333333   
1  0.307692  0.0     1.0                 1.0         0.857143      1.000000   
2  0.192308  0.0     1.0                 1.0         0.571429      0.166667   
3  0.192308  0.0     0.0                 0.5         0.285714      0.166667   
4  0.192308  0.0     0.0                 0.5         0.285714      0.000000   
5  0.307692  0.0     1.0                 1.0         0.857143      0.333333   
6  0.423077  0.0     1.0                 1.0         1.000000      0.166667   
7  0.038462  0.0     0.0                 0.0         0.285714      0.166667   
8  0.173077  0.0     0.0                 0.5         0.428571      0.000000   
9  0.211538  0.0     0.0                 1.0         0.571429      0.833333   

   genero_Feminino  genero_Masculino  genero_Outro  \
0              0.0               1.0           0.0   
1              0.0    

/tmp/ipykernel_15611/634232551.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["nivel_profissional"].fillna(df["nivel_profissional"].value_counts().index[0], inplace=True)


2. (0,5 pontos) Faça uma análise exploratória de dados.

  > Procure entender como os dados se comportam e as relações entre diferentes *features*.
  >
  > **Alguns fatores a serem considerados:** Quais as distribuição dos valores de cada atributos? Como é a correlação entre os diferentes atributos? Existe multicolinearidade entre algumas *features*?

In [ ]:
# TODO: Análise exploratória dos dados

df_proc.corr()

,idade,genero,pcd,gestor,nivel_profissional,tempo_exp_dados,tempo_exp_ti,salario,cor_raca_Amarela,cor_raca_Branca,cor_raca_Indígena,cor_raca_Outra,cor_raca_Parda,cor_raca_Unknown,cor_raca_Preta,estado_residencia_AC,estado_residencia_AL,estado_residencia_AP,estado_residencia_AM,estado_residencia_BA,estado_residencia_CE,estado_residencia_DF,estado_residencia_ES,estado_residencia_GO,estado_residencia_MA,estado_residencia_MT,estado_residencia_MS,estado_residencia_MG,estado_residencia_PR,estado_residencia_PB,estado_residencia_PA,estado_residencia_PE,estado_residencia_PI,estado_residencia_RN,estado_residencia_RS,estado_residencia_RJ,estado_residencia_RO,estado_residencia_SC,estado_residencia_SE,estado_residencia_SP,estado_residencia_TO,estado_residencia_nan,nivel_ensino_Doutorado ou Phd,nivel_ensino_Estudante,nivel_ensino_Graduação/Bacharelado,nivel_ensino_Mestrado,nivel_ensino_SemGrad,nivel_ensino_Unknown,nivel_ensino_Pós-graduação,area_formacao_Ciências Biológicas/ Farmácia/ Medicina/ Área da Saúde,area_formacao_Ciências Sociais,area_formacao_Computação / Engenharia de Software / Sistemas de Informação/ TI,area_formacao_Economia/ Administração / Contabilidade / Finanças/ Negócios,area_formacao_Estatística/ Matemática / Matemática Computacional/ Ciências Atuariais,area_formacao_Marketing / Publicidade / Comunicação / Jornalismo,area_formacao_Outra opção,area_formacao_Outras Engenharias,area_formacao_Química / Física,area_formacao_nan,situacao_trabalho_Empreendedor ou Empregado (CNPJ),situacao_trabalho_Empregado (CLT),situacao_trabalho_Estagiário,situacao_trabalho_Freelancer,situacao_trabalho_Prefiro não informar,situacao_trabalho_Servidor Público,situacao_trabalho_Vivo fora do Brasil e trabalho para empresa de fora do Brasil,situacao_trabalho_Vivo no Brasil e trabalho remoto para empresa de fora do Brasil,setor_Agronegócios,setor_Educação,setor_Entretenimento ou Esportes,setor_Filantropia/ONG's,setor_Finanças ou Bancos,setor_Indústria,setor_Internet/Ecommerce,setor_Marketing,setor_Outra Opção,setor_Seguros ou Previdência,setor_Setor Alimentício,setor_Setor Automotivo,setor_Setor Farmaceutico,setor_Setor Imobiliário/ Construção Civil,setor_Setor Público,setor_Setor de Energia,setor_Tech/Fábrica,setor_Telecomunicação,setor_Varejo,setor_Saude,setor_Consultoria,atuacao_dia_a_dia_AD/BI,atuacao_dia_a_dia_DS/ML/AI,atuacao_dia_a_dia_Engenharia,atuacao_dia_a_dia_Unknown,atuacao_dia_a_dia_nan,cargo_Analista de BI/BI Analyst,cargo_Analista de Dados/Data Analyst,cargo_Analista de Inteligência de Mercado/Market Intelligence,cargo_Analista de Negócios/Business Analyst,cargo_Analista de Suporte/Analista Técnico,cargo_Analytics Engineer,cargo_Cientista de Dados/Data Scientist,cargo_DBA/Administrador de Banco de Dados,cargo_Data Product Manager/ Product Manager (PM/APM/DPM/GPM/PO),cargo_Desenvolvedor/ Engenheiro de Software/ Analista de Sistemas,cargo_Diretor/VP,cargo_Economista,cargo_Engenheiro de Dados/Arquiteto de Dados/Data Engineer/Data Architect,cargo_Engenheiro de Machine Learning/ML Engineer/AI Engineer,cargo_Estatístico,cargo_Gerente/Head,cargo_Outra Opção,cargo_Outras Engenharias (não inclui dev),cargo_Professor/Pesquisador,cargo_Supervisor/Coordenador,"cargo_Sócio ou C-level (CEO, CDO, CIO, CTO etc)",cargo_Team Leader/Tech Leader
idade,1.000000,0.015834,0.020136,0.279067,0.385966,0.389281,0.361023,0.303226,0.022946,0.018406,-0.013648,0.009944,-0.009599,-0.019275,-0.026140,0.016272,-0.026297,0.005229,-0.021034,0.019709,0.004767,0.062520,-0.038740,0.011551,0.000206,-0.021802,-0.003754,-0.026310,-0.040329,-0.025723,-0.015171,-0.002732,0.001864,-0.036496,0.037869,0.025168,0.010745,-0.004309,-0.012363,-0.000868,0.005777,0.049524,0.127036,-0.291169,-0.166064,0.083970,-0.021897,-0.007198,0.250438,0.032206,-0.014125,0.039727,0.048517,0.001283,0.007403,0.020755,-0.112424,0.019889,-0.023034,0.066939,-0.039590,-0.195755,0.078423,-0.021007,0.161197,0.027777,-0.014486,-0.020086,0.044650,-0.013238,0.008687,-0.058781,0.033184,-0.024466,-0.023463,-0.022

3. (0,1 pontos) Divida o seu conjunto de treinamento em dois conjuntos: treinamento e validação. A proporção de divisão pode ser escolhida, mas pense no impacto que isso pode resultar em seu modelo.

  > **Dica:** Defina uma *seed* para seus resultados serem reprodutíveis.

In [24]:
# TODO: Divisão de treinamento e validação
df.insert(0, "bias", [1] * df.shape[0])
y = df["salario"].to_numpy()
print(y)
X = df.drop(["salario"], axis=1).to_numpy()
print(X)
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.3, random_state=42)

[27510. 23837. 21406. ...  5775.  3809.  2709.]
[[ 1.          0.21153846  0.33333333 ...  1.          0.
  11.        ]
 [ 1.          0.30769231  0.33333333 ...  0.          1.
  10.        ]
 [ 1.          0.19230769  0.         ...  0.          0.
  10.        ]
 ...
 [ 1.          0.25        0.         ...  0.          0.
   5.        ]
 [ 1.          0.30769231  0.         ...  0.          0.
   4.        ]
 [ 1.          0.15384615  0.         ...  0.          0.
   3.        ]]


## Regressão Linear

Esta parte do trabalho visa prever o valor do salário dos profissionais, descrita pelo atributo `salario`, com base em seus dados demográficos e informações de carreira.

O atributo alvo nesta etapa é o atributo `salario`. Portanto, não utilize os atributos `salario` e `faixa_salarial` como *features* dos seus modelos, apenas para o cálculo da função de custo durante o treinamento e a validação.

### Atividades

1. (0,1 pontos) Faça a seleção das features mais importantes para o treinamento da sua regressão linear, com base em sua análise exploratória.

In [ ]:
#TODO: Seleção de features.

# Seleção já realizada na etapa de pré-processamento

2. (1,75 pontos) Implemente e execute a regressão linear.

  Faça uma classe `LinearRegression` capaz de encontrar uma regressão utilizando dois métodos: **descida do gradiente** e a **equação normal**.

  Para a otimização por **descida do gradiente**, guarde o histórico da função de custo por época e retorno este histórico após a execução do ajuste. Além disso, utilize uma learning rate de `0.01` nesta etapa.

  > **Observações:** Inicie randomicamente sua regressão linear e lembre-se de não utilizar dados de teste no treinamento.

  Lembre-se que:

  > A hipótese da regressão linear é: $ h_\theta(x) = \theta^Tx$.
  >
  > E, o objetivo da regressão linear é minimizar a função de custo Root Mean Squared Error (RMSE):
  >
  > $$ J(\theta) = \frac{1}{2m} \sum_{i=1}^m (h_\theta(x^{(i)}) - y^{(i)})^2 $$
  >
  > Para minimizar o custo $J(\theta)$, o algoritmo de descida do gradiente realiza uma atualização nos parâmetros do modelo a cada iteração:
  >
  > $$\theta_j := \theta_j - \alpha \frac{\partial}{\partial\theta_j}J(\theta) = \theta_j - \alpha \frac{1}{m} \sum_{i=1}^m (h_\theta(x^{(i)}) - y^{(i)})x_j^{(i)} $$
  >
  > onde $j$ é o índice do parâmetro a ser atualizado (atualizando simultaneamente $\theta_j$ para todos os $j$) e $\alpha$ é a taxa de aprendizado.

In [25]:
# TODO: Regressão Linear. Implemente sua solução. Você não pode usar bibliotecas como scikit-learn, Keras/TensorFlow, ou PyTorch.

class LinearRegression:
    def __init__(self, X, y) -> None:
        self.X = X
        self.y = y
        pass

    def fit(self, method: str =None, **kwargs):
        if method == "gradient_descent":
            return self.fit_gradient_descent_solve(**kwargs)
        else:
            return self.fit_analytical_solve()

    def fit_analytical_solve(self):
        Xt = np.transpose(self.X)
        feature_prod = np.matmul(Xt, self.X)

        if (np.linalg.det(feature_prod) == 0):
            raise Exception("Matriz não invertível na solução analítica. Favor checar redundância dos features.")
        
        inv_prod = np.linalg.inv(feature_prod)
        z = np.matmul(inv_prod, Xt)
        theta = np.matmul(z, self.y)

        return theta
    
    def error_derivative(self, batch_size: int, theta: np.array):
        random_idx = np.random.choice(self.X.shape[0], size=batch_size)
        X_batch = self.X[random_idx]
        y_batch = self.y[random_idx]

        y_predict = np.matmul(X_batch, theta)
        error = np.sum((y_batch - y_predict) ** 2) / (2 * batch_size)
        error_d = X_batch.T @ (y_predict - y_batch) / batch_size

        return error, error_d

    def fit_gradient_descent_solve(self, epochs: int, batch_size: int, alpha: float = 0.01):
        theta = np.random.rand(self.X.shape[1])
        epochs_error = []

        for idx_epoch in range(epochs):
            for _ in range(self.X.shape[0] // batch_size):
                error, error_d = self.error_derivative(batch_size, theta)
                theta -= alpha * error_d
            epochs_error.append(error)
        
        return theta, epochs_error

In [30]:
# testando com regressão linear do scikit learn

from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
y_pred = regr.predict(X_validation)

print("coeff = \n", regr.coef_)
print("error = \n", mean_squared_error(y_validation, y_pred))

coeff = 
 [ 1.57043092e+11  1.41718621e+03 -6.38874909e+01  2.18712505e+02
  3.51105060e+15 -3.46656934e+03 -9.81250575e+01  5.73015028e+01
  4.53704283e+13  4.53704283e+13  4.53704283e+13  4.53704283e+13
  4.53704283e+13  4.53704283e+13  4.53704283e+13 -4.09581903e+14
 -4.09581903e+14 -4.09581903e+14 -4.09581903e+14 -4.09581903e+14
 -4.09581903e+14 -4.09581903e+14 -4.09581903e+14 -4.09581903e+14
 -4.09581903e+14 -4.09581903e+14 -4.09581903e+14 -4.09581903e+14
 -4.09581903e+14 -4.09581903e+14 -4.09581903e+14 -4.09581903e+14
 -4.09581903e+14 -4.09581903e+14 -4.09581903e+14 -4.09581903e+14
 -4.09581903e+14 -4.09581903e+14 -4.09581903e+14 -4.09581903e+14
 -4.09581903e+14 -4.09581903e+14  8.50180105e+14  8.50180105e+14
  8.50180105e+14  8.50180105e+14  8.19449888e+14  8.19449888e+14
  8.50180105e+14  3.29345934e+14  3.29345934e+14  3.29345934e+14
  3.29345934e+14  3.29345934e+14  3.29345934e+14  3.29345934e+14
  3.29345934e+14  3.29345934e+14  3.60076151e+14  2.26176314e+14
  2.26176314e+1

In [28]:
# X = np.random.rand(10000)
# X = X.reshape((10000, 1))
# X = np.hstack((np.ones(10000).reshape(10000, 1), X))
# y = 2* X[:,1] + 5

# lr = LinearRegression(X, y)

# theta = lr.fit()
# theta
# print(np.sum(X * theta - y)/10000)

lr = LinearRegression(X_train, y_train)

theta, epochs_error = lr.fit(method="gradient_descent", epochs=10000, batch_size=1000)

print((np.sum((X_train @ theta) - y_train) ** 2) / X_train.shape[0])
print(epochs_error)

44921117.6060942
[np.float64(37152308.7104624), np.float64(12228933.779253509), np.float64(32019255.732952464), np.float64(33142629.171817157), np.float64(30087425.7775392), np.float64(35244836.527093686), np.float64(24906910.125664044), np.float64(15141777.506666292), np.float64(19174966.67458568), np.float64(18824797.66155713), np.float64(18823630.356591444), np.float64(22404737.024015084), np.float64(20213742.646387313), np.float64(21208986.710726086), np.float64(17938266.258126713), np.float64(31173739.20438884), np.float64(24418936.628218677), np.float64(38665279.52902005), np.float64(19268931.852964707), np.float64(23877240.227535903), np.float64(16836263.834716916), np.float64(22456531.436539184), np.float64(28064636.43127116), np.float64(24424755.196013175), np.float64(14884170.112330196), np.float64(20520071.948339403), np.float64(27536245.501176048), np.float64(20035112.691009782), np.float64(25953160.6458795), np.float64(11425637.005383966), np.float64(10161695.619308788), n

In [29]:
X_train.shape

(2661, 116)

> **Quais foram suas conclusões? (1-2 parágrafos)**




*Escreva as conclusões aqui.*

3. (0,75 pontos) Experimente diferentes valores de *learning rates* para a descida de gradiente (GD). Plote o histórico da função de custo (função de custo vs. número de épocas) no conjunto de treinamento e no de validação para cada experimento.



In [ ]:
# TODO: Descida do gradiente (GD) com 3 diferentes learning rates.

> **Quais foram suas conclusões? (1-2 parágrafos)**




*Escreva as conclusões aqui.*

 4. (0,25 pontos) Compare o seu melhor experimento baseado em GD com a sua implementação da equação normal.

In [ ]:
# TODO: Compare as implementações

> **Quais foram suas conclusões? (1-2 parágrafos)**




*Escreva as conclusões aqui.*

5. (1,25 pontos) Implementa e execute duas regressões lineares utilizando sklearn:

  1. Execute uma regressão com `sklearn.linear_model.SGDRegressor`, que utiliza descida do gradiente minimizando uma função de custo regularizada.
  2. Execute uma regressão com `sklearn.linear_model.LinearRegression`, que utiliza o método da equação normal.

  Compare os resultados obtidos com as suas implementações (de GD e da equação normal).

  Ou seja, compare o `sklearn.linear_model.SGDRegressor` com a sua implementação da descida do gradiente e o `sklearn.linear_model.LinearRegression` com a sua implementação da equação normal.

  > **Observação:** Utilize uma learning rate igual a algum dos experimentos que já fez para realizar uma comparação justa.


In [ ]:
# TODO: Regressão Linear. Faça a regressão novamente, agora utilizando o sklearn.linear_model.SGDRegressor e sklearn.linear_model.LinearRegression.

> **Quais foram suas conclusões? (1-2 parágrafos)**




*Escreva as conclusões aqui.*

6. (0,4 ponto) Às vezes, precisamos de alguma função mais complexa para fazer uma boa predição. Elabore e avalie um modelo de Regressão Polinomial.

In [ ]:
# TODO: Regressão polinomial. Está liberado utilizar scikit-learn.

> **Quais foram suas conclusões? (1-2 parágrafos)**




*Escreva as conclusões aqui.*

## Regressão Logística

Esta parte do trabalho visa prever a faixa salarial dos profissionais, descrita pelo atributo `faixa_salarial`, com base em seus dados demográficos e informações de carreira. Portanto, não utilize os atributos `salario` e `faixa_salarial` como *features* dos seus modelos, apenas para o cálculo da função de custo durante o treinamento e a validação.

### Atividades

1. (0,1 pontos) Faça a seleção das features mais importantes para o treinamento da sua regressão logística, com base em sua análise exploratória.

In [ ]:
# TODO: Selecione de features.

2. (1,0 pontos) Faça uma Regressão Logística Multinomial. Ela é uma generalização da Regressão Logística para o caso em que queremos lidar com várias classes.

    Use a classe `sklearn.linear_model.SGDClassifier`, passando o parâmetro `loss="log_loss"` no construtor, para criar uma Regressão Logística a ser otimizada por descida do gradiente.

    Experimente diferentes valores de learning rate e avalie o impacto desse hiperparâmetro no treinamento do modelo.

In [ ]:
# TODO: Regressão logística multinomial. Utilize a classe sklearn.linear_model.SGDClassifier do scikit-learn.

> **Quais foram suas conclusões? (1-2 parágrafos)**




*Escreva as conclusões aqui.*

3. (0,75 ponto) Faça um gráfico da função de custo vs. número de épocas, em relação ao conjunto de treinamento e ao conjunto de validação. Também faça um gráfico, de mesma estrutura, descrevendo a acurácia vs. número de época.

    Como é o comportamento do modelo? O modelo está aprendendo?

In [ ]:
# TODO: Plote os gráficos de loss / acurácia vs. número de épocas. Você pode usar as bibliotecas scikit-learn, seaborn, matplotlib, etc.

> **Quais foram suas conclusões? (1-2 parágrafos)**




*Escreva as conclusões aqui.*

4. (0,9 pontos) Faça pelo menos 2 gráficos que visualizem as fronteiras de decisão entre duas *features*. Por exemplo, você pode fazer um gráfico da fronteira de decisão entre os atributos `tempo_exp_dados` e `satisfacao_empresa`, caso tenha utilizado essas *features*. Verifique se está fazendo o plot para atributos relevantes utilizados em seu modelo.

    > **Observação:** [Este tutorial](https://scikit-learn.org/stable/auto_examples/linear_model/plot_sgd_iris.html#sphx-glr-auto-examples-linear-model-plot-sgd-iris-py) do scikit-learn demonstra como extrair essas informações do `sklearn.linear_model.SGDClassifier`.

In [ ]:
# TODO: Faça a visualização das fronteiras de decisão. Você pode usar as bibliotecas scikit-learn, seaborn, matplotlib, etc.

> **Quais foram suas conclusões? (1-2 parágrafos)**




*Escreva as conclusões aqui.*

5. (0,75 pontos) Avalie os resultados do seu modelo, utilizando métricas de classificação que achar relevante. Plote a matriz de confusão e analise os resultados.

In [ ]:
# TODO: Avalie o seu modelo de classificação. Você pode usar as bibliotecas scikit-learn, seaborn, matplotlib, etc.

> **Quais foram suas conclusões? (1-2 parágrafos)**


*Escreva as conclusões aqui.*

## Submissão dos resultados no Kaggle

Para avaliar o desempenho do seu modelo, você deve submeter suas predições, no conjunto de teste, na competição [Tarefa #2 de MC886/MO444 2024S2](https://www.kaggle.com/t/20fa042c08f5453abd89d946468bd3de) do Kaggle.

A plataforma calculará automaticamente a métrica de avaliação e fornecerá um score.

> **Observação:** Não se preocupe apenas com o resultado do seu modelo. O seu score no placar não irá influenciar a sua nota nesta atividade :).

### Atividades

1. (1,0 pontos) Escolha **seus melhores modelos** para as duas tarefas feitas (regressão e classificação) e faça a predição dos atributos `salario` e `faixa_salarial` com base nos dados da pesquisa contidos no **conjunto de teste**.

  **Gere um arquivo de submissão, de tipo CSV, no formato adequado:** Gere uma linha por pessoa no conjunto de teste, com as duas colunas para os valores preditos, que devem estar nomeadas como **`salario`** e **`faixa_salarial`**.

  > Exemplo de arquivo (com o cabeçalho e conteúdo formatados):
  > ```
  > id, salario, faixa_salarial
  > 1, 0.0, "de R$ 0/mês a R$ 0/mês"
  > 2, 0.0, "de R$ 0/mês a R$ 0/mês"
  > 3, 0.0, "de R$ 0/mês a R$ 0/mês"
  > ```

  Submeta suas predições na competição [Tarefa #2 de MC886/MO444 2024S2](https://www.kaggle.com/t/20fa042c08f5453abd89d946468bd3de) no Kaggle.

  Conte para a gente! Qual foi o resultado que você obteve na competição? :)

*Conte aqui :)*

## Prazo

09 de outubro, quarta-feira, 23:59.

Política de penalidade para submissões atrasadas: Você não está sendo encorajada(o) a submeter o trabalho depois da data de submissão. Entretanto, caso isso aconteça, a nota será penalizada da seguinte forma:
- 10 de outubro, 23:59: nota * 0,75
- 11 de outubro, 23:59: nota * 0,5
- 12 de outubro, 23:59: nota * 0,25


## Envio

No Google Classroom, envie seu Jupyter Notebook, no formato RA1_RA2_2024s2_mc886mo444_tarefa_02.ipynb.

**Esta atividade NÃO é individual, deve ser realizada em dupla (grupo de duas pessoas).**
